In [2]:
!pip install pylast

In [3]:
import pandas as pd
import numpy as np
import os
import pylast
import time
import tqdm
import h5py
from scipy.sparse import coo_matrix, csr_matrix
import codecs
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [4]:
API_KEY = "dak62410481168654131dad23134624"  # this is a sample key
API_SECRET = "kfa69183915851kad51rr1131313131"
network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)

In [7]:
# borrowed from https://github.com/benfred/implicit, modified to read the file locally instead of downloading it.
def get_lastfm():
    """Returns a tuple of (artistids, userids, plays) where plays is a CSR matrix """
    
    filename = '../flaskapp/data/lastfm_360k.hdf5'

    with h5py.File(filename, 'r') as f:
        m = f.get('artist_user_plays')
        plays = csr_matrix((m.get('data'), m.get('indices'), m.get('indptr')))
        return np.array(f['artist']), np.array(f['user']), plays

In [8]:
# loading data
artists, users, plays = get_lastfm()

In [9]:
# return top n tags + their weight as a list
def lastfm_artist_to_top_n_tags(artist=str, n=int):
    #logging.basicConfig(level=logging.DEBUG)
    tag_weight = {}

    result = network.get_artist(artist).get_top_tags()

    for tag in result:
        tag_weight[str(tag.item.get_name())] = str(tag.weight) 

    tag_weight = {k: int(v) for k, v in tag_weight.items()}
    results = sorted(tag_weight.items(), key=lambda x: x[1], reverse=True)
    results = results[:n]
    return results

In [ ]:
start = time.time()
with tqdm.tqdm(total=len(artists)) as progress:
    with codecs.open('lastfm_artist_tags.csv', "w", "utf8") as o:
        for i in artists:
            a = i
            #a = a.strip()
            #print(a,end='')
            if(isinstance(a, str)):
                try: 
                    top = lastfm_artist_to_top_n_tags(a, n=10)
                    o.write("%s\t%s\n" % (a, str(top)))
                except:
                    pass
                    # if no tags, write nan
                    empty = np.nan 
                    o.write("%s\t%s\n" % (a, empty))
                top = ''
                progress.update(1)
        print('elapsed: '+str(time.time()-start))
logging.debug("process  in %0.2fs",  time.time() - start)

  0%|          | 63/292385 [01:14<91:28:03,  1.13s/it]DEBUG:pylast:artist.getTopTags
